In [59]:
import torch
import numpy as np
import mindspore
from mindspore import Tensor, dtype
from mindspore import save_checkpoint
from mindspore import Parameter
model = torch.load("beit_base_patch16_224_pt22k_ft22k.pth", map_location='cpu')['model']


In [60]:
model_keys = []
nums = 0.
for key in model.keys():
    if "num_batches_tracked" not in key:
        model_keys.append(key)
        nums += np.prod(model[key].shape)
    if "pos" in key:
        print(key)

blocks.0.attn.relative_position_bias_table
blocks.0.attn.relative_position_index
blocks.1.attn.relative_position_bias_table
blocks.1.attn.relative_position_index
blocks.2.attn.relative_position_bias_table
blocks.2.attn.relative_position_index
blocks.3.attn.relative_position_bias_table
blocks.3.attn.relative_position_index
blocks.4.attn.relative_position_bias_table
blocks.4.attn.relative_position_index
blocks.5.attn.relative_position_bias_table
blocks.5.attn.relative_position_index
blocks.6.attn.relative_position_bias_table
blocks.6.attn.relative_position_index
blocks.7.attn.relative_position_bias_table
blocks.7.attn.relative_position_index
blocks.8.attn.relative_position_bias_table
blocks.8.attn.relative_position_index
blocks.9.attn.relative_position_bias_table
blocks.9.attn.relative_position_index
blocks.10.attn.relative_position_bias_table
blocks.10.attn.relative_position_index
blocks.11.attn.relative_position_bias_table
blocks.11.attn.relative_position_index


In [61]:
weights = []
prefix = "model."
for key in model_keys:
    name2weight = {}
    if "bn" in key or "norm" in key or "ln" in key:
        if "weight" in key:
            name2weight["name"] = prefix + key.replace(".weight", ".gamma")
        elif "bias" in key:
            name2weight["name"] = prefix + key.replace(".bias", ".beta")
        elif "mean" in key:
            name2weight["name"] = prefix + key.replace("running_mean", "moving_mean")
        elif "var" in key:
            name2weight["name"] = prefix + key.replace("running_var", "moving_variance")
        
        name2weight["data"] = Parameter(Tensor(model[key].numpy(), dtype.float32),requires_grad=True) 
        weights.append(name2weight)
    elif "qkv" in key:
        key_q = prefix +  key.replace("qkv", "q")
        key_k = prefix +  key.replace("qkv", "k")
        key_v = prefix + key.replace("qkv", "v")
        shape = model[key].shape[0]//3
        weight = Parameter(Tensor(model[key].numpy(), dtype.float32),requires_grad=True) 
        weight_q = weight[:shape]
        weight_k = weight[shape:shape*2]
        weight_v = weight[shape*2:]
        weights.append({"name":key_q, "data": weight_q})
        weights.append({"name":key_k, "data": weight_k})
        weights.append({"name":key_v, "data": weight_v})
    elif "q_bias" in key:
        name2weight["name"] = prefix + key.replace("q_bias", "q.bias")
        name2weight["data"] = Parameter(Tensor(model[key].numpy(), dtype.float32),requires_grad=True)
        weights.append(name2weight)
    elif "v_bias" in key:
        name2weight["name"] = prefix + key.replace("v_bias", "v.bias")
        name2weight["data"] = Parameter(Tensor(model[key].numpy(), dtype.float32),requires_grad=True)
        weights.append(name2weight)
    elif "head" in key:
        # we don't need classfier layer
        pass
    else:
        dd = dtype.float32
        if "relative_position_index" in key:
            dd = dtype.int32
        weight = Parameter(Tensor(model[key].numpy(), dd),requires_grad=True)
        key = prefix + key
        weights.append({"name": key, "data": weight})

(1000, 768)
(1000,)


In [62]:
weights_out = []
for weight in weights:
    weight_out = {}
    name = weight["name"]
    weight_out["name"] = name
    weight_out["data"] = weight["data"]
    weights_out.append(weight_out)
    print(weight_out["name"], weight_out['data'].shape)

model.cls_token (1, 1, 768)
model.patch_embed.proj.weight (768, 3, 16, 16)
model.patch_embed.proj.bias (768,)
model.blocks.0.gamma_1 (768,)
model.blocks.0.gamma_2 (768,)
model.blocks.0.norm1.gamma (768,)
model.blocks.0.norm1.beta (768,)
model.blocks.0.attn.q.bias (768,)
model.blocks.0.attn.v.bias (768,)
model.blocks.0.attn.relative_position_bias_table (732, 12)
model.blocks.0.attn.relative_position_index (197, 197)
model.blocks.0.attn.q.weight (768, 768)
model.blocks.0.attn.k.weight (768, 768)
model.blocks.0.attn.v.weight (768, 768)
model.blocks.0.attn.proj.weight (768, 768)
model.blocks.0.attn.proj.bias (768,)
model.blocks.0.norm2.gamma (768,)
model.blocks.0.norm2.beta (768,)
model.blocks.0.mlp.fc1.weight (3072, 768)
model.blocks.0.mlp.fc1.bias (3072,)
model.blocks.0.mlp.fc2.weight (768, 3072)
model.blocks.0.mlp.fc2.bias (768,)
model.blocks.1.gamma_1 (768,)
model.blocks.1.gamma_2 (768,)
model.blocks.1.norm1.gamma (768,)
model.blocks.1.norm1.beta (768,)
model.blocks.1.attn.q.bias (768,

In [63]:
save_checkpoint(weights_out, "beit_base_patch16_224_pt22k_ft22k.ckpt")

In [64]:
len(weights_out)

235

In [65]:
import torch
import numpy as np
import mindspore
from mindspore import Tensor, dtype
from mindspore import save_checkpoint
from mindspore import Parameter
model = torch.load("beit_base_patch16_224_pt22k_ft22kto1k.pth", map_location='cpu')['model']
weights = []
prefix = "model."
for key in model_keys:
    name2weight = {}
    if "bn" in key or "norm" in key or "ln" in key:
        if "weight" in key:
            name2weight["name"] = prefix + key.replace(".weight", ".gamma")
        elif "bias" in key:
            name2weight["name"] = prefix + key.replace(".bias", ".beta")
        elif "mean" in key:
            name2weight["name"] = prefix + key.replace("running_mean", "moving_mean")
        elif "var" in key:
            name2weight["name"] = prefix + key.replace("running_var", "moving_variance")
        
        name2weight["data"] = Parameter(Tensor(model[key].numpy(), dtype.float32),requires_grad=True) 
        weights.append(name2weight)
    elif "qkv" in key:
        key_q = prefix +  key.replace("qkv", "q")
        key_k = prefix +  key.replace("qkv", "k")
        key_v = prefix + key.replace("qkv", "v")
        shape = model[key].shape[0]//3
        weight = Parameter(Tensor(model[key].numpy(), dtype.float32),requires_grad=True) 
        weight_q = weight[:shape]
        weight_k = weight[shape:shape*2]
        weight_v = weight[shape*2:]
        weights.append({"name":key_q, "data": weight_q})
        weights.append({"name":key_k, "data": weight_k})
        weights.append({"name":key_v, "data": weight_v})
    elif "q_bias" in key:
        name2weight["name"] = prefix + key.replace("q_bias", "q.bias")
        name2weight["data"] = Parameter(Tensor(model[key].numpy(), dtype.float32),requires_grad=True)
        weights.append(name2weight)
    elif "v_bias" in key:
        name2weight["name"] = prefix + key.replace("v_bias", "v.bias")
        name2weight["data"] = Parameter(Tensor(model[key].numpy(), dtype.float32),requires_grad=True)
        weights.append(name2weight)
    else:
        dd = dtype.float32
        if "relative_position_index" in key:
            dd = dtype.int32
        weight = Parameter(Tensor(model[key].numpy(), dd),requires_grad=True)
        key = prefix + key
        weights.append({"name": key, "data": weight})
weights_out = []
for weight in weights:
    weight_out = {}
    name = weight["name"]
    weight_out["name"] = name
    weight_out["data"] = weight["data"]
    weights_out.append(weight_out)
    print(weight_out["name"], weight_out['data'].shape)
save_checkpoint(weights_out, "beit_base_patch16_224_pt22k_ft22kto1k.ckpt")

model.cls_token (1, 1, 768)
model.patch_embed.proj.weight (768, 3, 16, 16)
model.patch_embed.proj.bias (768,)
model.blocks.0.gamma_1 (768,)
model.blocks.0.gamma_2 (768,)
model.blocks.0.norm1.gamma (768,)
model.blocks.0.norm1.beta (768,)
model.blocks.0.attn.q.bias (768,)
model.blocks.0.attn.v.bias (768,)
model.blocks.0.attn.relative_position_bias_table (732, 12)
model.blocks.0.attn.relative_position_index (197, 197)
model.blocks.0.attn.q.weight (768, 768)
model.blocks.0.attn.k.weight (768, 768)
model.blocks.0.attn.v.weight (768, 768)
model.blocks.0.attn.proj.weight (768, 768)
model.blocks.0.attn.proj.bias (768,)
model.blocks.0.norm2.gamma (768,)
model.blocks.0.norm2.beta (768,)
model.blocks.0.mlp.fc1.weight (3072, 768)
model.blocks.0.mlp.fc1.bias (3072,)
model.blocks.0.mlp.fc2.weight (768, 3072)
model.blocks.0.mlp.fc2.bias (768,)
model.blocks.1.gamma_1 (768,)
model.blocks.1.gamma_2 (768,)
model.blocks.1.norm1.gamma (768,)
model.blocks.1.norm1.beta (768,)
model.blocks.1.attn.q.bias (768,

In [66]:
len(weights_out)

235